## Neural Style Transfer

Neural style transfer takes two inputs: a content image (the image whose structure we want to preserve) and a style image (the image from which we extract artistic style). The output is a new image that blends the content of the first with the style of the second.